In [0]:
# 重启
# !kill -9 -1 
# 切换到自己drive的数据存储目录
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab_Notebooks/Employee_Resignation
!ls
# install and import package
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import seaborn as sns

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab_Notebooks/Employee_Resignation
predict_lr.csv	README.md  Resignation_Rate.ipynb  test.csv  train.csv


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Load data 加载数据

In [0]:
train_data = pd.read_csv('train.csv', index_col='user_id')
train_y = train_data['Attrition'].replace(['No', 'Yes'], [0, 1])
train_x = train_data.drop(columns=['Attrition', 'EmployeeNumber', 
                  'Over18', 'EmployeeCount'])
test_data = pd.read_csv('test.csv', index_col='user_id')
test_x = test_data.drop(columns=['EmployeeNumber', 'Over18', 
                  'EmployeeCount'])

## Explore Data 数据探索

In [0]:
pd.set_option('display.max_columns', None)  # 显示所有列
print('查看数据信息：列名、非空个数、类型等')
print(train_data.info())
print('-' * 30)
print('查看数据摘要')
print(train_data.describe())
print('-' * 30)
print('查看离散数据分布')
print(train_data.describe(include=['O']))
print('-' * 30)
print('查看前5条数据')
print(train_data.head())
print('-' * 30)
print('查看后5条数据')
print(train_data.tail())

## 数据预处理

In [0]:
# 使用one hot编码进行特征值处理 略低于使用数只替换，0.828
# 特征转换，字符型转成不同列
dvec = DictVectorizer(sparse=False)
train_x = dvec.fit_transform(train_x.to_dict(orient='record'))
test_x = dvec.transform(test_x.to_dict(orient='record'))


In [0]:
# 将离散值转换为数值类型的数据预处理 0.829
def extract_features(df):
  # BusinessTravel
  businesstravel_dict = {'Non-Travel':0, 'Travel_Rarely':1, 'Travel_Frequently':2}
  df['BusinessTravel'] = df['BusinessTravel'].map(lambda x: businesstravel_dict[x])
  # Department
  department_dict = {'Sales':0, 'Research & Development':1, 'Human Resources':2}
  df['Department'] = df['Department'].map(lambda x: department_dict[x])
  # EducationField
  educationfield_dict = {'Life Sciences':0, 'Medical':1, 'Marketing':2, 'Technical Degree':3, 'Human Resources':4, 'Other':5}
  df['EducationField'] = df['EducationField'].map(lambda x: educationfield_dict[x])
  # Gender
  gender_dict = {'Male':0, 'Female': 1}
  df['Gender'] = df['Gender'].map(lambda x: gender_dict[x])
  # JobRole
  jobrole_dict = {'Sales Executive':0, 
                  'Research Scientist':1, 
                  'Laboratory Technician':2, 
                  'Manufacturing Director':3, 
                  'Healthcare Representative':4,
                  'Manager':5, 
                  'Sales Representative':6,
                  'Research Director':7,
                  'Human Resources':8
                  }
  df['JobRole'] = df['JobRole'].map(lambda x: jobrole_dict[x])
  # MaritalStatus
  maritalstatus_dict = {'Single':0, 'Married':1, 'Divorced':2}
  df['MaritalStatus'] = df['MaritalStatus'].map(lambda x: maritalstatus_dict[x])
  # OverTime
  overtime_dict = {'Yes':0, 'No':1}
  df['OverTime'] = df['OverTime'].map(lambda x: overtime_dict[x])
  return df
train_x = extract_features(train_x)
test_x = extract_features(test_x)

In [0]:
# 采用min-max规范化
min_max_scaler = preprocessing.MinMaxScaler()
train_x = min_max_scaler.fit_transform(train_x)
test_x = min_max_scaler.transform(test_x)

## 模型训练与预测

In [0]:
# 使用逻辑回归 获得predict_proba的概率 0.829
# class_weight='balanced'
lr = LogisticRegression()
lr.fit(train_x,train_y)
print('LR准确率(基于训练集)： %.4lf' % lr.score(train_x, train_y))
predict_lr = lr.predict_proba(test_x)

df1 = test_data
df1['Attrition'] =predict_lr[:,1]
df1 = df1['Attrition']
df1.to_csv('predict_lr.csv')

LR准确率(基于训练集)： 0.8946


In [9]:
# 使用LightGBM进行回归预测 0.838
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
model = lgb.LGBMClassifier()
parameters = {
        'max_depth': [7, 15, 30],
        'learning_rate': [0.01, 0.1, 0.5],
        'feature_fraction': [0.6, 0.8, 0.95],
        'bagging_fraction': [0.6, 0.8, 0.95],
        'bagging_freq': [2, 5, 8],
        'lambda_l1': [0, 0.1, 0.5],
        'lambda_l2': [0, 10, 40],
        'cat_smooth': [1, 15, 35]
        }
gsearch = GridSearchCV(model, param_grid=parameters, scoring='accuracy', cv=3)
gsearch.fit(train_x, train_y)

best_parameters = gsearch.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

print('lgb准确率(基于训练集)： %.4lf' % gsearch.score(train_x, train_y))
# 预测
predict_lgb = gsearch.predict_proba(test_x)
# 输出
df1 = test_data
df1['Attrition'] =predict_lgb[:,1]
df1 = df1['Attrition']
df1.to_csv('predict_lgb.csv')

	bagging_fraction: 0.95
	bagging_freq: 8
	cat_smooth: 1
	feature_fraction: 0.6
	lambda_l1: 0
	lambda_l2: 40
	learning_rate: 0.5
	max_depth: 15
lgb准确率(基于训练集)： 0.9898
